# Basic analysis using xarray

In [2]:
import numpy as np
import xarray as xr
import xrft

import dask.array as da
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt

In [19]:
! wget https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz
! gunzip gistemp1200_GHCNv4_ERSSTv5.nc.gz 
! mv gistemp1200_GHCNv4_ERSSTv5.nc ~/Tomfoolery/data/ERSSTv5.nc

--2021-07-22 13:09:54--  https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz
Resolving data.giss.nasa.gov (data.giss.nasa.gov)... 129.164.128.233
Connecting to data.giss.nasa.gov (data.giss.nasa.gov)|129.164.128.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24898858 (24M) [application/x-gzip]
Saving to: ‘gistemp1200_GHCNv4_ERSSTv5.nc.gz’

gistemp1200_GHCNv4_ 100%[===================>]  23.75M  3.57MB/s    in 5.6s    

2021-07-22 13:09:59 (4.27 MB/s) - ‘gistemp1200_GHCNv4_ERSSTv5.nc.gz’ saved [24898858/24898858]



In [21]:
ds = xr.open_dataset('~/Tomfoolery/data/ERSSTv5.nc')
ds

<xarray.Dataset>
Dimensions:      (lat: 90, lon: 180, nv: 2, time: 1698)
Coordinates:
  * lat          (lat) float32 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon          (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time         (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-06-15
Dimensions without coordinates: nv
Data variables:
    time_bnds    (time, nv) datetime64[ns] ...
    tempanomaly  (time, lat, lon) float32 ...
Attributes:
    title:        GISTEMP Surface Temperature Analysis
    institution:  NASA Goddard Institute for Space Studies
    source:       http://data.giss.nasa.gov/gistemp/
    Conventions:  CF-1.6
    history:      Created 2021-07-09 10:49:43 by SBBX_to_nc 2.0 - ILAND=1200,...

In [25]:
temp = ds['tempanomaly']
temp

<xarray.DataArray 'tempanomaly' (time: 1698, lat: 90, lon: 180)>
[27507600 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-06-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

## Detrend dataset

In [32]:
def detrend_dataarray(darray, dim='time', deg=1):
    # needed to workaround xarray's check with zero dimensions
    # https://github.com/pydata/xarray/issues/3575
    if len(darray[dim]) == 0:
        return darray
    
    p = darray.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(darray[dim], p.polyfit_coefficients)
    return darray - fit

detrend_dataarray(temp);

<xarray.DataArray (time: 1698, lat: 90, lon: 180)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [ 0.10191583,  0.10191583,  0.10191583, ...,  0.10191583,
          0.10191583,  0.10191583],
        [ 0.10191583,  0.10191583,  0.10191583, ...,  0.10191583,
          0.10191583,  0.10191583],
        [ 0.10191583,  0.10191583,  0.10191583, ...,  0.10191583,
          0.10191583,  0.10191583]],

       [[-2.65761132, -2.65761132, -2.65761132, ..., -2.65761132,
         -2.65761132, -2.65761132],
        [-2.65761132, -2.65761132, -2.65761132, ..., -2.65761132,
         -2.65761132, -2.65761132],
        [-2.65761132, -2.65761132, -2.65761132, ..., -2.65761132,
         -2.65761132, -2.65761132],
        ...,
        [-0.24996777, -0.24996777, -0.24996777, ..., -0.24996777,
         -0.24996777, -0.24996777],
        [-0.24996777, -0.24996777, -0.24996777, ..., -0.24996777,
         -0.24996777, -0.24996777],
        [-0.24996777, -0.24996777, -0.24996777, ..., -0.24996777,
         -0.24996777, -0.24996777]]])
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-06-15

In [33]:
def remove_climatology(darray, timedim='time', groupby='dayofyear'):
    gb = darray.groupby(timedim + '.' + groupby)
    clim = gb.mean(dim=timedim)
    return gb - clim
remove_climatology(temp)

/Users/andrewbrettin/opt/anaconda3/envs/pangeo/lib/python3.8/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'tempanomaly' (time: 1698, lat: 90, lon: 180)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [ 1.2071211 ,  1.2071211 ,  1.2071211 , ...,  1.2071211 ,
          1.2071211 ,  1.2071211 ],
        [ 1.2071211 ,  1.2071211 ,  1.2071211 , ...,  1.2071211 ,
          1.2071211 ,  1.2071211 ],
        [ 1.2071211 ,  1.2071211 ,  1.2071211 , ...,  1.2071211 ,
          1.2071211 ,  1.2071211 ]],

       [[-2.2479591 , -2.2479591 , -2.2479591 , ..., -2.2479591 ,
         -2.2479591 , -2.2479591 ],
        [-2.2479591 , -2.2479591 , -2.2479591 , ..., -2.2479591 ,
         -2.2479591 , -2.2479591 ],
        [-2.2479591 , -2.2479591 , -2.2479591 , ..., -2.2479591 ,
         -2.2479591 , -2.2479591 ],
        ...,
        [ 1.1919698 ,  1.1919698 ,  1.1919698 , ...,  1.1919698 ,
          1.1919698 ,  1.1919698 ],
        [ 1.1919698 ,  1.1919698 ,  1.1919698 , ...,  1.1919698 ,
          1.1919698 ,  1.1919698 ],
        [ 1.1919698 ,  1.1919698 ,  1.1919698 , ...,  1.1919698 ,
          1.1919698 ,  1.1919698 ]]], dtype=float32)
Coordinates:
  * lat        (lat) float32 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon        (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time       (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-06-15
    dayofyear  (time) int64 15 46 75 106 136 167 197 ... 15 46 74 105 135 166